In [ ]:
# =========================
# Setup (Colab safe)
# =========================
%pip -q install opencv-python-headless gradio plotly pytransform3d numpy matplotlib

import os, json, glob, random, io, base64
from dataclasses import dataclass
import numpy as np
import cv2
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Optional: pytransform3d for nicer 3D frames (we'll also provide a fallback)
try:
    from pytransform3d.transform_manager import TransformManager
    from pytransform3d.rotations import matrix_from_euler_xyz
    _HAS_P3D = True
except Exception:
    _HAS_P3D = False

# =========================
# IO helpers
# =========================
class IO:
    @staticmethod
    def ensure_dir(path: str):
        os.makedirs(path, exist_ok=True)

    @staticmethod
    def save_json(obj, path: str):
        with open(path, "w") as f:
            json.dump(obj, f, indent=2)

    @staticmethod
    def load_images_from_dir(dir_path: str, exts=(".jpg", ".jpeg", ".png")):
        paths = []
        for e in exts:
            paths.extend(glob.glob(os.path.join(dir_path, f"*{e}")))
        return sorted(paths)

    @staticmethod
    def imread_rgb(path: str):
        bgr = cv2.imread(path, cv2.IMREAD_COLOR)
        if bgr is None:
            return None
        return cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

    @staticmethod
    def imwrite_rgb(path: str, rgb):
        bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
        cv2.imwrite(path, bgr)

# =========================
# Chessboard & camera math
# =========================
class Board:
    """
    Chessboard definition.
    pattern_size = (cols, rows) = number of INNER corners (e.g., (9,6))
    square_size = length of each square in meters (e.g., 0.025)
    """
    @staticmethod
    def object_points(pattern_size, square_size):
        cols, rows = pattern_size
        objp = np.zeros((rows * cols, 3), np.float32)
        grid = np.mgrid[0:cols, 0:rows].T.reshape(-1, 2)
        objp[:, :2] = grid * square_size
        return objp  # (N,3)

class Img:
    @staticmethod
    def find_corners(gray, pattern_size):
        flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE
        ok, corners = cv2.findChessboardCorners(gray, pattern_size, flags)
        if not ok:
            return False, None
        # Refine
        criteria = (cv2.TermCriteria_EPS + cv2.TermCriteria_MAX_ITER, 30, 1e-3)
        corners_refined = cv2.cornerSubPix(
            gray,
            corners,
            winSize=(11, 11),
            zeroZone=(-1, -1),
            criteria=criteria,
        )
        return True, corners_refined

    @staticmethod
    def to_gray(rgb):
        return cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

class Calib:
    """
    Runs OpenCV calibration and returns intrinsics + per-image extrinsics.
    """
    @staticmethod
    def calibrate(image_paths, pattern_size=(9, 6), square_size=0.025):
        obj_pts = []   # 3D points in world (board) coords
        img_pts = []   # 2D points in image
        imsize = None

        objp = Board.object_points(pattern_size, square_size)

        valid_paths = []
        for p in image_paths:
            rgb = IO.imread_rgb(p)
            if rgb is None:
                continue
            gray = Img.to_gray(rgb)
            if imsize is None:
                imsize = (gray.shape[1], gray.shape[0])
            ok, corners = Img.find_corners(gray, pattern_size)
            if ok:
                obj_pts.append(objp.copy())
                img_pts.append(corners)
                valid_paths.append(p)

        if len(obj_pts) < 5:
            raise ValueError("Too few valid chessboard detections. Need at least ~5 images.")

        # Calibrate
        flags = 0
        ret, K, D, rvecs, tvecs = cv2.calibrateCamera(
            objectPoints=obj_pts,
            imagePoints=img_pts,
            imageSize=imsize,
            cameraMatrix=None,
            distCoeffs=None,
            flags=flags
        )

        # Reprojection error
        per_view_errors = []
        for i in range(len(obj_pts)):
            proj, _ = cv2.projectPoints(obj_pts[i], rvecs[i], tvecs[i], K, D)
            err = cv2.norm(img_pts[i], proj, cv2.NORM_L2) / len(proj)
            per_view_errors.append(float(err))
        mean_err = float(np.mean(per_view_errors))

        # Extrinsics per image as R (3x3) and t (3,)
        extrinsics = []
        for rv, tv in zip(rvecs, tvecs):
            R, _ = cv2.Rodrigues(rv)
            extrinsics.append({"R": R.tolist(), "t": tv.flatten().tolist()})

        result = {
            "ret_rms": float(ret),
            "mean_reproj_error": mean_err,
            "image_size": {"width": imsize[0], "height": imsize[1]},
            "K": K.tolist(),
            "D": D.flatten().tolist(),
            "pattern_size": {"cols": pattern_size[0], "rows": pattern_size[1]},
            "square_size_m": float(square_size),
            "num_views": len(valid_paths),
            "per_view_errors": per_view_errors,
            "valid_paths": valid_paths,
            "extrinsics": extrinsics
        }
        return result

    @staticmethod
    def undistort(rgb, K, D, keep_size=True):
        h, w = rgb.shape[:2]
        K = np.array(K, dtype=np.float64)
        D = np.array(D, dtype=np.float64)
        newK, _ = cv2.getOptimalNewCameraMatrix(K, D, (w, h), alpha=0)
        und = cv2.undistort(rgb, K, D, None, newK)
        if keep_size:
            und = cv2.resize(und, (w, h), interpolation=cv2.INTER_AREA)
        return und

    @staticmethod
    def solve_pnp_for_image(pattern_size, square_size, corners, K, D):
        objp = Board.object_points(pattern_size, square_size)
        ok, rvec, tvec = cv2.solvePnP(objp, corners, K, D, flags=cv2.SOLVEPNP_ITERATIVE)
        if not ok:
            return None, None
        R, _ = cv2.Rodrigues(rvec)
        return R, tvec

class Overlay:
    """
    Utilities to draw axes on detected board and produce sample overlays.
    """
    @staticmethod
    def draw_axes(rgb, K, D, rvec, tvec, axis_len=0.05):
        # Define 3D axis points
        pts3d = np.float32([
            [0, 0, 0],
            [axis_len, 0, 0],
            [0, axis_len, 0],
            [0, 0, axis_len],
        ]).reshape(-1, 3)
        pts2d, _ = cv2.projectPoints(pts3d, rvec, tvec, K, D)
        pts2d = pts2d.reshape(-1, 2).astype(int)

        img = rgb.copy()
        origin = tuple(pts2d[0])
        xpt = tuple(pts2d[1])
        ypt = tuple(pts2d[2])
        zpt = tuple(pts2d[3])

        # X=red, Y=green, Z=blue
        cv2.line(img, origin, xpt, (255, 0, 0), 3)
        cv2.line(img, origin, ypt, (0, 255, 0), 3)
        cv2.line(img, origin, zpt, (0, 0, 255), 3)
        return img

    @staticmethod
    def make_sample_overlays(image_paths, K, D, pattern_size, square_size, max_images=8):
        show_paths = random.sample(image_paths, min(max_images, len(image_paths)))
        out_images = []
        for p in show_paths:
            rgb = IO.imread_rgb(p)
            gray = Img.to_gray(rgb)
            ok, corners = Img.find_corners(gray, pattern_size)
            if not ok:
                continue
            ok, rvec, tvec = cv2.solvePnP(
                Board.object_points(pattern_size, square_size),
                corners, np.array(K, np.float64), np.array(D, np.float64),
                flags=cv2.SOLVEPNP_ITERATIVE
            )
            if not ok:
                continue
            img_axes = Overlay.draw_axes(rgb, np.array(K, np.float64), np.array(D, np.float64), rvec, tvec, axis_len=square_size*3)
            out_images.append((p, img_axes))
        return out_images

# =========================
# 3D Visualization of camera poses
# =========================
class Render:
    @staticmethod
    def camera_frames_from_extrinsics(extrinsics):
        """
        Returns camera centers and optical axes in board/world coordinates.
        Each extrinsic: world_T_cam = [R|t] maps cam->world? In OpenCV:
        rvec,tvec map object(world)->image with world->cam = [R|t].
        So camera center in world is C = -R^T t
        Camera z-axis in world is R^T * [0,0,1]
        """
        centers = []
        z_axes = []
        for ex in extrinsics:
            R = np.array(ex["R"])
            t = np.array(ex["t"]).reshape(3, 1)
            C = (-R.T @ t).flatten()               # camera center in world
            z_axis = (R.T @ np.array([0, 0, 1.0])).flatten()
            centers.append(C)
            z_axes.append(z_axis)
        return np.array(centers), np.array(z_axes)

    @staticmethod
    def plot_poses_plotly(extrinsics, square_size=0.025, board_size=(9,6)):
        centers, z_axes = Render.camera_frames_from_extrinsics(extrinsics)

        # Board corners outline for reference
        cols, rows = board_size
        w = (cols-1) * square_size
        h = (rows-1) * square_size
        bx = [0, w, w, 0, 0]
        by = [0, 0, h, h, 0]
        bz = [0, 0, 0, 0, 0]

        fig = go.Figure()
        # Board outline
        fig.add_trace(go.Scatter3d(x=bx, y=by, z=bz, mode='lines', name='Board'))

        # Camera centers
        fig.add_trace(go.Scatter3d(
            x=centers[:,0], y=centers[:,1], z=centers[:,2],
            mode='markers', name='Cameras', marker=dict(size=5)
        ))

        # Camera z-axis rays
        scale = square_size * 3
        for c, z in zip(centers, z_axes):
            p2 = c + z * scale
            fig.add_trace(go.Scatter3d(x=[c[0], p2[0]], y=[c[1], p2[1]], z=[c[2], p2[2]],
                                       mode='lines', name='optical_axis',
                                       line=dict(width=4)))

        fig.update_layout(
            width=800, height=500,
            scene=dict(
                xaxis_title="X (m)", yaxis_title="Y (m)", zaxis_title="Z (m)",
                aspectmode="data"
            ),
            title="Estimated Camera Poses w.r.t. Chessboard"
        )
        return fig

    @staticmethod
    def plot_poses_matplotlib(extrinsics, square_size=0.025, board_size=(9,6)):
        centers, z_axes = Render.camera_frames_from_extrinsics(extrinsics)
        cols, rows = board_size
        w = (cols-1) * square_size
        h = (rows-1) * square_size

        fig = plt.figure(figsize=(7,5))
        ax = fig.add_subplot(111, projection='3d')
        # Board outline
        bx = [0, w, w, 0, 0]
        by = [0, 0, h, h, 0]
        bz = [0, 0, 0, 0, 0]
        ax.plot(bx, by, bz, lw=2, label="Board")

        ax.scatter(centers[:,0], centers[:,1], centers[:,2], s=10, label="Cameras")
        scale = square_size * 3
        for c, z in zip(centers, z_axes):
            p2 = c + z * scale
            ax.plot([c[0], p2[0]],[c[1], p2[1]],[c[2], p2[2]], lw=2)

        ax.set_xlabel("X (m)"); ax.set_ylabel("Y (m)"); ax.set_zlabel("Z (m)")
        ax.legend()
        ax.set_title("Estimated Camera Poses w.r.t. Chessboard")
        plt.tight_layout()
        return fig


In [ ]:
import gradio as gr

# Default working directory (as requested)
BASE_DIR = "/content"
IMG_DIR = os.path.join(BASE_DIR, "images")
OUT_JSON = os.path.join(BASE_DIR, "calibration.json")
IO.ensure_dir(IMG_DIR)

def save_uploads(files):
    """
    Save uploaded images to /content/images and return a message + count.
    """
    saved = 0
    for f in files:
        # Gradio provides a tempfile path in f.name
        data = open(f.name, "rb").read()
        # Normalize extension to .jpg
        name = os.path.basename(f.name)
        if not (name.lower().endswith(".jpg") or name.lower().endswith(".jpeg")):
            name = os.path.splitext(name)[0] + ".jpg"
        out_path = os.path.join(IMG_DIR, name)
        with open(out_path, "wb") as w:
            w.write(data)
        saved += 1
    return f"Saved {saved} image(s) to {IMG_DIR}"

def run_calibration(cols, rows, square_size_m):
    """
    Runs OpenCV calibration using images found in /content/images.
    Writes results to calibration.json and returns a text summary.
    """
    paths = IO.load_images_from_dir(IMG_DIR, exts=(".jpeg", ".jpg"))
    if len(paths) < 5:
        return f"Need at least ~5 images; found {len(paths)} in {IMG_DIR}. Add more and retry."
    pattern = (int(cols), int(rows))
    try:
        result = Calib.calibrate(paths, pattern_size=pattern, square_size=float(square_size_m))
    except Exception as e:
        return f"Calibration failed: {e}"

    IO.save_json(result, OUT_JSON)

    # Short human-readable summary
    K = np.array(result["K"])
    D = np.array(result["D"])
    msg = [
        f"Saved calibration to: {OUT_JSON}",
        f"Views used: {result['num_views']}",
        f"Image size: {result['image_size']}",
        f"RMS reprojection error: {result['ret_rms']:.4f}",
        f"Mean per-view error: {result['mean_reproj_error']:.4f}",
        f"K (intrinsics):\n{K}",
        f"D (distortion): {D}",
    ]
    return "\n".join(msg)

def show_poses_and_overlays(max_overlays, use_plotly=True):
    """
    Loads calibration.json, generates:
    - 3D pose figure (plotly if available, else matplotlib static PNG)
    - Up to N images with axes overlays
    - One undistortion preview (before/after)
    """
    if not os.path.exists(OUT_JSON):
        return None, None, "Run calibration first."

    with open(OUT_JSON, "r") as f:
        calib = json.load(f)

    K = np.array(calib["K"], dtype=np.float64)
    D = np.array(calib["D"], dtype=np.float64)
    cols = int(calib["pattern_size"]["cols"])
    rows = int(calib["pattern_size"]["rows"])
    square_size = float(calib["square_size_m"])
    extrinsics = calib["extrinsics"]
    valid_paths = calib["valid_paths"]

    # 3D poses plot
    if use_plotly:
        fig = Render.plot_poses_plotly(extrinsics, square_size=square_size, board_size=(cols, rows))
    else:
        fig = Render.plot_poses_matplotlib(extrinsics, square_size=square_size, board_size=(cols, rows))

    # Overlays (axes on sample images)
    overlays = Overlay.make_sample_overlays(valid_paths, K, D, (cols, rows), square_size, max_images=int(max_overlays))
    gallery = []
    for p, img in overlays:
        # Convert to BGR for encoding (cv2.imencode expects BGR)
        bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        ok, buf = cv2.imencode(".jpg", bgr)
        if ok:
            gallery.append((p, buf.tobytes()))
    if not gallery:
        gallery = None

    # Undistort preview (use the first valid path)
    und_preview = None
    if len(valid_paths) > 0:
        sample = IO.imread_rgb(valid_paths[0])
        und = Calib.undistort(sample, K, D, keep_size=True)
        # Stack side-by-side
        s = cv2.cvtColor(sample, cv2.COLOR_RGB2BGR)
        u = cv2.cvtColor(und, cv2.COLOR_RGB2BGR)
        pad = np.ones((s.shape[0], 10, 3), dtype=np.uint8) * 255
        side = np.hstack([s, pad, u])
        ok, buf = cv2.imencode(".jpg", side)
        if ok:
            und_preview = buf.tobytes()

    return fig, gallery, und_preview

# =========================
# Build Gradio UI
# =========================
with gr.Blocks(title="Camera Calibration (OpenCV + Gradio)", fill_height=True) as demo:
    gr.Markdown("## Camera Calibration (OpenCV + Gradio)\nUpload chessboard images, run calibration, and visualize results.\n\n- Images are saved to **/content/images**\n- Results saved to **/content/calibration.json**\n- Pattern size = INNER corners (e.g., 9×6)\n")

    with gr.Row():
        uploader = gr.Files(label="Upload .jpeg / .jpg images", file_types=["image"])
        upload_btn = gr.Button("Save to /content/images")

    upload_msg = gr.Textbox(label="Upload Log", interactive=False)

    with gr.Accordion("Calibration Parameters", open=True):
        with gr.Row():
            cols = gr.Number(value=9, label="Inner corners (cols)", precision=0)
            rows = gr.Number(value=6, label="Inner corners (rows)", precision=0)
            sq = gr.Number(value=0.025, label="Square size (meters)")

        run_btn = gr.Button("Run Calibration")
        calib_out = gr.Textbox(label="Calibration Output", lines=12)

    with gr.Accordion("Visualizations", open=True):
        with gr.Row():
            max_ov = gr.Slider(3, 10, value=6, step=1, label="Max overlay images")
            use_plotly = gr.Checkbox(value=True, label="Interactive 3D (Plotly)")

        pose_plot = gr.Plot(label="Camera Poses (3D)")
        gallery = gr.Gallery(label="Sample images with axes", columns=3, height=300)
        und_preview = gr.Image(label="Undistort Preview (left=original, right=undistorted)")

        viz_btn = gr.Button("Generate Visualizations")

    # Wiring
    upload_btn.click(fn=save_uploads, inputs=[uploader], outputs=[upload_msg])
    run_btn.click(fn=run_calibration, inputs=[cols, rows, sq], outputs=[calib_out])
    viz_btn.click(fn=show_poses_and_overlays, inputs=[max_ov, use_plotly],
                  outputs=[pose_plot, gallery, und_preview])

demo.launch(share=False, inline=True, debug=False)
